In [ ]:
# Cell 1
import os, sys, pickle
sys.path.append('../src')

# Clean models to avoid old pickle loading
os.makedirs('../models', exist_ok=True)
for f in ('../models/hmm_model.pkl','../models/q_table.pkl'):
    if os.path.exists(f):
        os.remove(f)
print("✓ Old models removed")

from utils import load_words, preprocess_words
from hmm_model import HangmanHMM
from hangman_env import HangmanEnv
from rl_agent import QLearningAgent

# Cell 2
print("\nLoading corpus...")
words = load_words('../data/corpus.txt')
words = [w for w in words if w.isalpha()]
train_words = preprocess_words(words)

# Cell 3
print("\nTraining HMM...")
hmm = HangmanHMM()
hmm.train(train_words)

with open('../models/hmm_model.pkl', 'wb') as f:
    pickle.dump(hmm, f)
print("✓ HMM saved")

# Quick verification of smart fallback presence
print("Has _smart_fallback:", hasattr(hmm, '_smart_fallback'))

# Cell 4 - Quick 10-game sanity test
print("\nQuick test (10 games)...")
env = HangmanEnv(train_words)
agent = QLearningAgent(hmm)
wins = 0
for i in range(10):
    state = env.reset()
    done = False
    steps = 0
    while not done and steps < 50:
        probs = hmm.predict_probabilities(state['masked_word'], state['guessed_letters'])
        action = agent.choose_action(state, probs)
        if action is None:
            break
        state, reward, done, info = env.step(action)
        steps += 1
    wins += 1 if env.won else 0

print(f"Sanity success: {wins}/10")


✓ Old models removed

Loading corpus...
Loaded 50000 words, 49979 alphabetic
Filtered to 48446 words (length 3-15)

Training HMM...

=== PATTERN MATCHING HMM ===
Corpus: 48446 words | Lengths: 3..15
=== READY ===

✓ HMM saved
Has _smart_fallback: True

Quick test (10 games)...
Sanity success: 9/10
